# Install dependencies

In [1]:
!pip install --upgrade torch torchtext

Requirement already up-to-date: torch in /usr/local/lib/python3.7/dist-packages (1.8.1+cu101)
Requirement already up-to-date: torchtext in /usr/local/lib/python3.7/dist-packages (0.9.1)


In [2]:
!pip install nltk swifter livelossplot

     |████████████████████████████████| 634kB 6.7MB/s 
     |████████████████████████████████| 296kB 22.4MB/s 
     |████████████████████████████████| 583kB 22.2MB/s 
     |████████████████████████████████| 112kB 35.2MB/s 
     |████████████████████████████████| 48.5MB 63kB/s 
     |████████████████████████████████| 17.2MB 211kB/s 
     |████████████████████████████████| 1.3MB 47.1MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 133kB 54.1MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 204kB 53.5MB/s 
     |████████████████████████████████| 3.1MB 37.7MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 296kB 48.1MB/s 
     |████████████████████████████████| 143kB 51.9MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
  Created wheel for swifter: filename=swifter-1.0.7-cp37-none-any.whl size=13980 sha256=a5fc3600249571364f0c6266b

In [3]:
!pip install allennlp

     |████████████████████████████████| 604kB 4.9MB/s 
     |████████████████████████████████| 2.1MB 10.0MB/s 
     |████████████████████████████████| 266kB 26.2MB/s 
     |████████████████████████████████| 122kB 18.5MB/s 
     |████████████████████████████████| 1.2MB 21.9MB/s 
     |████████████████████████████████| 2.0MB 51.7MB/s 
     |████████████████████████████████| 133kB 48.8MB/s 
     |████████████████████████████████| 133kB 49.7MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 163kB 47.9MB/s 
     |████████████████████████████████| 3.3MB 45.4MB/s 
     |████████████████████████████████| 870kB 13.9MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 7.4MB 50.3MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388791 sha256=c21bc3860a63e3383f0fc82d04371fd2192d22b52edd7bdccb3672472721f4d

# Initialize

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm.notebook import tqdm

import pandas as pd
pd.options.display.max_colwidth = 150

torch.manual_seed(1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
from joblib import Memory


CACHE_DIR = 'cache'
if not os.path.isdir(CACHE_DIR):
    os.mkdir(CACHE_DIR)
    
MEMORY = Memory(CACHE_DIR, verbose=0) 

# Load and preprocess the corpus annotated with sentiment

In [7]:
import torchtext
from torchtext.utils import download_from_url, extract_archive

datasets = {
    'yelp_review_polarity_csv' : 'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbNUpYQ2N3SGlFaDg',
    'yelp_review_full_csv' : 'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbZlU4dXhHTFhZQU0',
    'amazon_review_polarity_csv' : 'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbaW12WVVZS2drcnM',
    'amazon_review_full_csv': 'https://drive.google.com/uc?export=download&id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA'
}

DATA_PATH = 'data'
if not os.path.isdir(DATA_PATH):
    os.mkdir(DATA_PATH)

dataset_name = 'yelp_review_polarity_csv'

dataset_tar = download_from_url(datasets[dataset_name], root=DATA_PATH)
extracted_files = extract_archive(dataset_tar)
extracted_files

yelp_review_polarity_csv.tar.gz: 166MB [00:02, 76.2MB/s]


['/content/data/yelp_review_polarity_csv/readme.txt',
 '/content/data/yelp_review_polarity_csv/test.csv',
 '/content/data/yelp_review_polarity_csv/train.csv']

In [8]:
pd.read_csv(extracted_files[1], header=None, names=['label', 'text']).head()

,label,text
0,2,"Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now,..."
1,1,Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with...
2,2,"Friendly staff, same starbucks fair you get anywhere else. Sometimes the lines can get long."
3,1,"The food is good. Unfortunately the service is very hit or miss. The main issue seems to be with the kitchen, the waiters and waitresses are often..."
4,2,Even when we didn't have a car Filene's Basement was worth the bus trip to the Waterfront. I always find something (usually I find 3-4 things and ...


In [9]:
from nltk.tokenize import TweetTokenizer
import swifter
from torchtext.vocab import Vocab
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from pathlib import Path


def create_vocab(dataset):  
    counter = Counter()
    for inst in dataset:
        counter.update(inst)

    return Vocab(counter, min_freq=1)


def vectorize_dataset(dataset, vocab):
    result = []
    for example in dataset:
        result.append([vocab[t] for t in example])
    return result


@MEMORY.cache
def preprocess_dataset(data_path, max_train, max_test, max_dev, random_state=1981):
    columns = ['label', 'text']
    test_df = pd.read_csv(Path(data_path) / 'test.csv', header=None, names=columns)
    train_df = pd.read_csv(Path(data_path) / 'train.csv', header=None, names=columns)

    test_df = test_df.sample(n=max_test, random_state=random_state)
    train_df = train_df.sample(n=max_dev + max_train, random_state=random_state)
    
    dev_df = train_df[max_train:]
    train_df = train_df[:max_train]
    
    tokenizer = TweetTokenizer()    
    def ling_preprocess(text):
        # TODO: add lemmatization
        # TODO: add bigrams
        return [e.lower() for e in tokenizer.tokenize(text)]
    
    train_text_preproc = train_df.text.swifter.apply(ling_preprocess)
    test_text_preproc = test_df.text.swifter.apply(ling_preprocess)
    dev_text_preproc = dev_df.text.swifter.apply(ling_preprocess)
    
    vocab = create_vocab(train_text_preproc)
    
    train_token_ids = vectorize_dataset(train_text_preproc, vocab)
    test_token_ids = vectorize_dataset(test_text_preproc, vocab)
    dev_token_ids = vectorize_dataset(dev_text_preproc, vocab)
    
    label_encoder = LabelEncoder()
    train_label_ids = label_encoder.fit_transform(train_df.label)
    test_label_ids = label_encoder.transform(test_df.label)
    dev_label_ids = label_encoder.transform(dev_df.label)
    
    train_dataset = list(zip(train_label_ids, train_token_ids))
    dev_dataset = list(zip(dev_label_ids, dev_token_ids))
    test_dataset = list(zip(test_label_ids, test_token_ids))

    return (vocab, label_encoder), (train_dataset, test_dataset, dev_dataset), (train_df, test_df, dev_df)

In [ ]:
vocab_label_enc, vectorized_datasets, text_datasets = preprocess_dataset(data_path=Path(DATA_PATH) / dataset_name, 
                                                                        max_train=4000, 
                                                                        max_test=4000,
                                                                        max_dev=1000)

vocab, label_encoder = vocab_label_enc
train_dataset, test_dataset, dev_dataset = vectorized_datasets
train_text_dataset, test_text_dataset, dev_text_dataset = text_datasets

VOCAB_SIZE = len(vocab)
NUM_LABELS = label_encoder.classes_.shape[0]

print('Vocab size:', VOCAB_SIZE)
print('Number of labels:', NUM_LABELS)

In [ ]:
train_dataset[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


def make_bow_vector_count(dataset, vocab_len):
    result = torch.zeros((len(dataset), vocab_len), dtype=torch.int32)
    for i in range(len(dataset)):
        for word in dataset[i]:
            result[i, word] += 1
        
    return result


def train_vectorizer(dataset, vocab_len):
    bow = make_bow_vector_count(dataset, vocab_len)
    vectorizer = TfidfTransformer()
    
    vectorizer.fit(bow)
    return vectorizer


def make_bow_vector_tfidf(dataset, vectorizer):
    bow = make_bow_vector_count(dataset, vectorizer.idf_.shape[0])
    return torch.Tensor(vectorizer.transform(bow).todense())

In [ ]:
VECTORIZER = train_vectorizer(train_dataset, VOCAB_SIZE)

# Logistic regression model in pytorch

## Creating model

In [ ]:
class LogisticRegressionBow(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super().__init__()
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

As we can see below, there is no anything special about nn.Linear and F.linear. They just implement formula: $XW^T+b$ 

In [ ]:
nn.Linear??

In [ ]:
F.linear??

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
model = LogisticRegressionBow(NUM_LABELS, VOCAB_SIZE)
for param in model.parameters():
    print(param)
    
print('Number of parameters', count_parameters(model))

In [ ]:
# As we can see the model is not trained and outputs garbage.

n_top = 3
with torch.no_grad():
    for sample, text in zip(test_dataset[:n_top], test_text_dataset.iloc[:n_top].text.tolist()):
        label, instance = sample
        bow_vec = make_bow_vector_tfidf([instance], VECTORIZER)
        log_probs = model(bow_vec)
        print('Text: ', text[:150], '...', '| Prediction:', log_probs)
        print('======================')

print('Features')
good_word = 'good'
print(f'{good_word} : ', next(model.parameters())[:, vocab[good_word]])
bad_word = 'nasty'
print(f'{bad_word} : ', next(model.parameters())[:, vocab[bad_word]])

## Training model: dummy example

In [ ]:
# Dummy stochastic gradient descent

N_EPOCHS = 4
LEARNING_RATE = 0.1

loss_function = nn.NLLLoss() # Negative log likelihood loss
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(N_EPOCHS):
    for sample in tqdm(train_dataset):
        label, instance = sample
        
        # Make our BOW vector and also we must wrap the target in a
        # Tensor as an integer. For example, if the target is NEGATIVE, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to NEGATIVE
        bow_vec = make_bow_vector_tfidf([instance], VECTORIZER)
        target = torch.LongTensor([label])

        # Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        
        # Run forward pass.
        log_probs = model(bow_vec)

        # Compute the loss, gradients, and update the parameters
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [ ]:
# We see that our model learned something reasonable
n_top = 3
with torch.no_grad():
    for sample, text in zip(test_dataset[:n_top], test_text_dataset.iloc[:n_top].text.tolist()):
        label, instance = sample
        bow_vec = make_bow_vector_tfidf([instance], VECTORIZER)
        log_probs = model(bow_vec)
        print('Text: ', text[:150], '...', '| Prediction:', log_probs)
        print('======================')

print('Features')
good_word = 'good'
print(f'{good_word} : ', next(model.parameters())[:, vocab[good_word]])
bad_word = 'nasty'
print(f'{bad_word} : ', next(model.parameters())[:, vocab[bad_word]])

## Train model using DataLoader

In [ ]:
from torch.utils.data import DataLoader


def generate_batch_bow(batch):
    features = make_bow_vector_tfidf([inst for _, inst in batch], VECTORIZER)
    labels = torch.LongTensor([label for label, _ in batch])    
    return labels, features

In [ ]:
BATCH_SIZE = 50
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                              collate_fn=generate_batch_bow, num_workers=1)

In [ ]:
N_EPOCHS = 4
LEARNING_RATE = 0.1

model = LogisticRegressionBow(NUM_LABELS, VOCAB_SIZE)
model.train()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss()


for epoch in range(N_EPOCHS):
    for labels, features in tqdm(train_dataloader):       
        model.zero_grad()
        log_probs = model(features)
        loss = loss_function(log_probs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
n_top = 3
with torch.no_grad():
    for sample, text in zip(test_dataset[:n_top], test_text_dataset.iloc[:n_top].text.tolist()):
        label, instance = sample
        bow_vec = make_bow_vector_tfidf([instance], VECTORIZER)
        log_probs = model(bow_vec)
        print('Text: ', text[:150], '...', '| Prediction:', log_probs)
        print('======================')

print('Features')
good_word = 'good'
print(f'{good_word} : ', next(model.parameters())[:, vocab[good_word]])
bad_word = 'nasty'
print(f'{bad_word} : ', next(model.parameters())[:, vocab[bad_word]])

In [ ]:
from torch.utils.data import DataLoader


def test(test_dataset, model, batch_size, device=None):
    model.eval()
    loss = 0
    acc = 0
    data_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=generate_batch_bow)
    for sample in tqdm(data_loader):
        labels, features = sample
        if device is not None:
            labels = labels.to(device)
            features = features.to(device)
        with torch.no_grad():
            output = model(features)
            loss += loss_function(output, labels).item()
            acc += (output.argmax(1) == labels).sum().item()
    
    return loss / len(test_dataset), acc / len(test_dataset)

In [ ]:
loss, acc = test(test_dataset, model, batch_size=200)
print(f'Test loss: {loss: .4f}')
print(f'Test accuracy: {acc: .4f}')

# Neural network model

In [ ]:
class FFNNBow(nn.Module):
    def __init__(self, vocab_size, num_classes, hidden_layer_size=5):
        super().__init__()
        self.hidden_layer = nn.Linear(vocab_size, hidden_layer_size)
        self.relu = torch.nn.ReLU()
        self.output_layer = nn.Linear(hidden_layer_size, num_classes)
    
    def forward(self, features):
        intermid = self.hidden_layer(features)
        intermid = self.relu(intermid)
        intermid = self.output_layer(intermid)
        output = F.log_softmax(intermid, dim=1)
        return output

In [ ]:
model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)

for param in model.parameters():
    print(param)
    
print('Number of parameters', count_parameters(model))

In [ ]:
from torch.utils.data import DataLoader


def train_model_epoch(train_dataset, model, optimizer, bs):
    model.train()
    train_loss = 0
    train_acc = 0
    data = DataLoader(train_dataset, batch_size=bs, 
                      shuffle=True, collate_fn=generate_batch_bow, 
                      num_workers=0)
    
    for sample in tqdm(data):
        labels, features = sample
        labels = labels.to(device)
        features = features.to(device)
        
        optimizer.zero_grad()
        output = model(features)
        loss = loss_function(output, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    return train_loss / len(train_dataset)


def train(train_dataset, test_dataset, model, optimizer, n_epochs, bs, pred_bs):
    print('Training...')
    for epoch in range(n_epochs):
        train_loss = train_model_epoch(train_dataset, model, optimizer, bs=bs)
        print(f'Train loss {train_loss:.4f}')
    print('Finished training.')

    print('Evaluating...')
    test_loss, test_acc = test(test_dataset, model, batch_size=pred_bs, device=device)
    print(f'Test loss: {test_loss:.4f}')
    print(f'Test accuracy: {test_acc: .4f}')

In [ ]:
N_EPOCHS = 4
BATCH_SIZE = 50
LEARNING_RATE = 0.01
PRED_BATCH_SIZE = 200

loss_function = nn.NLLLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
train(train_dataset, test_dataset, 
      model, optimizer, 
      n_epochs=N_EPOCHS, 
      bs=BATCH_SIZE, 
      pred_bs=PRED_BATCH_SIZE)

## Using better gradient descent algorithm (Adam)

In [ ]:
N_EPOCHS = 4
BATCH_SIZE = 50
LEARNING_RATE = 0.01
PRED_BATCH_SIZE = 200

model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)
loss_function = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
train(train_dataset, test_dataset, 
      model, optimizer, 
      n_epochs=N_EPOCHS, 
      bs=BATCH_SIZE, 
      pred_bs=PRED_BATCH_SIZE)

## Adding regularization with L2 weight norm

In [ ]:
N_EPOCHS = 4
BATCH_SIZE = 50
LEARNING_RATE = 0.01
PRED_BATCH_SIZE = 200

model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)
loss_function = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
train(train_dataset, test_dataset, 
      model, optimizer, 
      n_epochs=N_EPOCHS, 
      bs=BATCH_SIZE, 
      pred_bs=PRED_BATCH_SIZE)

## Learning rate annealing and early stopping

In [ ]:
from livelossplot import PlotLosses
from torch.optim.lr_scheduler import ReduceLROnPlateau

from livelossplot.outputs import MatplotlibPlot, ExtremaPrinter


def train_epoch_adv(train_dataset, 
                    model, 
                    optimizer,
                    collate_fn, 
                    loss_fn,
                    scheduler=None, 
                    bs=50):
    model.train()
    data_loader = DataLoader(train_dataset, batch_size=bs, 
                             shuffle=True, collate_fn=collate_fn)
    train_loss = 0
    for cls, tensors in tqdm(data_loader):
        if type(tensors) is not tuple:
            tensors = (tensors, )
            
        optimizer.zero_grad()
        tensors, cls = tuple(t.to(device) for t in tensors), cls.to(device)
        output = model(*tensors)
        loss = loss_fn(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if scheduler is not None and not isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step()

    return train_loss / len(train_dataset)


def test_adv(dataset, model, collate_fn, loss_fn, bs):
    model.eval()
    data_loader = DataLoader(dataset, batch_size=bs, collate_fn=collate_fn)
    test_loss = 0
    acc = 0
    for cls, tensors in tqdm(data_loader):
        if type(tensors) is not tuple:
            tensors = (tensors, )
            
        tensors, cls = tuple(t.to(device) for t in tensors), cls.to(device)
        with torch.no_grad():
            output = model(*tensors)
            test_loss += loss_fn(output, cls).item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(dataset), acc / len(dataset)


def train_adv(train_dataset, dev_dataset, 
              model, optimizer, 
              collate_fn, loss_fn, 
              bs, pred_bs, 
              scheduler=None, 
              draw=False,
              tol=-1,
              eps=1e-8):
    liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(5,3)), ExtremaPrinter()])
    
    print('Training...')
    best_loss = float('inf')
    tol_iter = 0
    for epoch in range(N_EPOCHS):
        train_loss = train_epoch_adv(train_dataset, model, optimizer, 
                                     collate_fn=collate_fn, loss_fn=loss_fn, 
                                     scheduler=scheduler, bs=bs)
        val_loss, val_acc = test_adv(dev_dataset, model, 
                                     collate_fn=collate_fn, loss_fn=loss_fn, 
                                     bs=pred_bs)   

        if scheduler is not None and isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(val_loss)

        if draw:
            liveloss.update({'train_loss' : train_loss,
                             'val_loss' : val_loss, 
                             'val_acc' : val_acc})
            liveloss.draw()

        if tol >= 0:
            if best_loss - val_loss > eps:
                best_loss = val_loss
            else:
                tol_iter += 1
                if tol_iter > tol:
                    break
    
    print('Finished training.')

In [ ]:
N_EPOCHS = 15
BATCH_SIZE = 50
PRED_SIZE = 100
LEARNING_RATE = 0.01

model = FFNNBow(VOCAB_SIZE, NUM_LABELS, 5).to(device)
loss_function = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss().to(device)

try:
    train_adv(train_dataset, dev_dataset, 
              model, optimizer, 
              generate_batch_bow, 
              loss_function, 
              bs=BATCH_SIZE, 
              pred_bs=PRED_SIZE, 
              scheduler=None, 
              draw=True)
except KeyboardInterrupt:
    pass

test_loss, test_acc = test_adv(test_dataset, model, 
                               generate_batch_bow, 
                               loss_function, PRED_SIZE)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
# We will use early stopping for preventing the overfitting.
# In the real-world scentario, we would like to also track the best model and restore it, when
# the validation performance was at maximum.

N_EPOCHS = 15
BATCH_SIZE = 50
PRED_SIZE = 100
LEARNING_RATE = 0.01

model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)
loss_function = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss().to(device)

try:
    train_adv(train_dataset, dev_dataset, 
            model, optimizer, 
            generate_batch_bow, 
            loss_function, 
            bs=BATCH_SIZE, 
            pred_bs=PRED_SIZE, 
            scheduler=None, 
            draw=True, 
            tol=1)
except KeyboardInterrupt:
    pass


test_loss, test_acc = test_adv(test_dataset, model, 
                               generate_batch_bow, 
                               loss_function, PRED_SIZE)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
# Training with smaller learning rate can help to fit model better. But
# if we train with small learning rates we also need smaller batch size (so we use batch_size=5). 
# This all results in a very slow training process.

N_EPOCHS = 5
BATCH_SIZE = 5
PRED_SIZE = 100
LEARNING_RATE = 0.0001

model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)
loss_function = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss().to(device)

try:
    train_adv(train_dataset, 
            dev_dataset, 
            model, 
            optimizer, 
            generate_batch_bow, 
            loss_function, 
            bs=BATCH_SIZE, 
            pred_bs=PRED_SIZE, 
            scheduler=None, 
            draw=True, 
            tol=1)
except KeyboardInterrupt:
    pass


test_loss, test_acc = test_adv(test_dataset, model, 
                               generate_batch_bow, 
                               loss_function, PRED_SIZE)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
# It is better to use learning rate scheduler. We start from a big learning rate 
# for faster convergence and closer to the end of the process we reduce LR to a small value

N_EPOCHS = 15
BATCH_SIZE = 50
PRED_SIZE = 100
LEARNING_RATE = 0.01

model = FFNNBow(VOCAB_SIZE, NUM_LABELS).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1)
loss_function = nn.NLLLoss().to(device)

train_adv(train_dataset, dev_dataset, 
          model, optimizer, 
          generate_batch_bow, loss_function, 
          bs=BATCH_SIZE, pred_bs=PRED_SIZE, 
          scheduler=scheduler, draw=True, tol=5)

test_loss, test_acc = test_adv(test_dataset, model, 
                               generate_batch_bow, 
                               loss_function, PRED_SIZE)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

## Model with embeddings

In [ ]:
class FFNNEmbedding(nn.Module):
    def __init__(self, vocab_size, num_class, embed_dim):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
    
def generate_batch_emb(batch):
    texts = [inst for _, inst in batch]
    offsets = [0] + [len(text) for text in texts]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    labels = torch.tensor([label for label, _ in batch])
    texts =  torch.cat(tuple(torch.tensor(e) for e in texts))
    return labels, (texts, offsets)

In [ ]:
EMBED_DIM = 20

model = FFNNEmbedding(VOCAB_SIZE, NUM_LABELS, EMBED_DIM)
print('Number of parameters', count_parameters(model))
print(count_parameters(model.fc))
print(count_parameters(model.embedding))

In [ ]:
N_EPOCHS = 40
EMBED_DIM = 20
BATCH_SIZE = 50
PRED_SIZE = 100
LEARNING_RATE = 0.001

model = FFNNEmbedding(VOCAB_SIZE, NUM_LABELS, EMBED_DIM).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=1)
loss_function = nn.CrossEntropyLoss().to(device)

train_adv(train_dataset, dev_dataset, 
          model, optimizer, 
          generate_batch_emb, loss_function, 
          bs=BATCH_SIZE, pred_bs=PRED_SIZE, 
          scheduler=scheduler, draw=True, tol=3)

test_loss, test_acc = test_adv(test_dataset, model, 
                               generate_batch_emb, 
                               loss_function, PRED_SIZE)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_acc}')

# Other tips to incorporate in your deep learning pipeline

1. Gradient normalization and clipping.
2. Stochastic weight averaging / exponential weight averaging.
3. Batch normalization in the model and other types of normalizaiton.
4. Keep track of the best model on the validation set and restore it when you are doing annealing and at the end.

# Allennlp



## Simple model

In [ ]:
from allennlp.models import BasicClassifier
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, LabelField
from allennlp.data import Token, Instance

from nltk.tokenize import TweetTokenizer

from typing import Iterable


class ClassificationCsvReader(DatasetReader):
    def __init__(self, text_col, label_col, indexer, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tokenizer = TweetTokenizer()
        self.token_indexers = {'tokens' : indexer}
        self._label_col = label_col
        self._text_col = text_col

    def _read(self, file_path: str) -> Iterable[Instance]:
        dataframe = pd.read_csv(file_path, header=None, 
                                names=[self._label_col, self._text_col])
        
        for i in dataframe.index:
            yield self.text_to_instance(dataframe.loc[i, self._text_col], 
                                        str(dataframe.loc[i, self._label_col]))

    def text_to_instance(self, text, target):
        text_field = TextField([Token(e.lower()) for e in self.tokenizer.tokenize(text)],
                                   self.token_indexers)
        label_field = LabelField(target)
        fields = {'tokens': text_field, 'label': label_field}
        return Instance(fields)

In [ ]:
from itertools import islice
from pathlib import Path

from allennlp.data.token_indexers import SingleIdTokenIndexer


indexer = SingleIdTokenIndexer()

text_col = 'text'
label_col = 'label'
reader = ClassificationCsvReader(label_col=label_col, text_col=text_col, indexer=indexer)

train_dataset = reader.read(Path(DATA_PATH) / dataset_name / 'train.csv')
test_dataset = reader.read(Path(DATA_PATH) / dataset_name / 'test.csv')

train_dataset = list(tqdm(islice(train_dataset, 4000)))
test_dataset = list(tqdm(islice(test_dataset, 4000)))

print(next(iter(train_dataset)))

In [ ]:
from allennlp.data.vocabulary import Vocabulary

vocab = Vocabulary.from_instances(train_dataset, min_count={"tokens": 1})
print('Vocabulary size:', vocab.get_vocab_size('tokens'))

In [ ]:
from allennlp.training import GradientDescentTrainer
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2vec_encoders import BagOfEmbeddingsEncoder

EMBEDDING_DIM = 20

embedder = Embedding(embedding_dim=EMBEDDING_DIM, vocab=vocab)
text_field_embedder = BasicTextFieldEmbedder({'tokens': embedder})
encoder = BagOfEmbeddingsEncoder(embedding_dim=EMBEDDING_DIM, averaged=True)
model = BasicClassifier(vocab, 
                        text_field_embedder=text_field_embedder, 
                        seq2vec_encoder=encoder, 
                        dropout=0.1, 
                        num_labels=vocab.get_vocab_size('labels'))
model = model.to(device)
print('Number of paramters:', count_parameters(model))
model

In [ ]:
from allennlp.data.data_loaders import SimpleDataLoader
from allennlp.training.learning_rate_schedulers import ReduceOnPlateauLearningRateScheduler


optimizer = optim.Adam(
    model.parameters(),
    lr=0.01,
    weight_decay=1e-6,
)

train_data_loader = SimpleDataLoader(
    instances=train_dataset, batch_size=50
)
train_data_loader.index_with(vocab)

test_data_loader = SimpleDataLoader(
    instances=test_dataset, batch_size=200
)
test_data_loader.index_with(vocab)


lr_scheduler = ReduceOnPlateauLearningRateScheduler(optimizer, factor=0.5, patience=1)

trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=test_data_loader,
    num_epochs=40,
    cuda_device=device,
    learning_rate_scheduler=lr_scheduler,
    num_gradient_accumulation_steps=1,
    serialization_dir=None,
    patience=3
)

try:
    metrics = trainer.train()
except KeyboardInterrupt:
    pass

In [ ]:
from allennlp.training.util import evaluate

evaluate(model, test_data_loader, cuda_device=device)

# Quick tip: In case of insufficient CUDA memory error

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()